In [5]:
import pandas as pd
import numpy as np
import math
import time

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [6]:
vic_data = pd.read_csv('../kennedy/victoria.csv')

In [7]:
# Constant Variables 
TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

POWER = 300
CAPACITY = 580
EFFICIENCY = 0.9
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

CHARGE = -1
DISCHARGE = 1

In [8]:
# percentile.exc from excel != np.percentile (np.percentile == percentile.inc from excel)
# code taken from https://stackoverflow.com/questions/38596100/python-equivalent-of-excels-percentile-exc

def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - ser_sorted.iat[rank_l]) * (rank - rank_l)

In [9]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    df['decision1'] = 0
    
    # I removed the first row because first row of vic is 00:00:00, 
    # which is the last period from the previous year
    df = df.drop([0], axis=0) 
    
    return df

In [41]:
def algorithm3(df):
    """ Finds optimal charge and discharge time across the dataset """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = df.iloc[i:i+LOOKAHEAD][PRICE]
            ser = pd.Series(thelist)

            
            if (df.at[i,PRICE] <= quantile_exc(ser,CHARGING_PERCENTILE)) & (df.at[i,PRICE] < thelist.mean()):
                df.at[i,'decision1'] = CHARGE

            if (df.at[i,PRICE] >= quantile_exc(ser,DISCHARGING_PERCENTILE)) & (df.at[i,PRICE] > thelist.mean()):
                df.at[i,'decision1'] = DISCHARGE
                
        else:
            thelist = df.iloc[i:][PRICE]
            ser = pd.Series(thelist)

            if (df.at[i,PRICE] <= ser.quantile(CHARGING_PERCENTILE)) & (df.at[i,PRICE] < thelist.mean()):
                df.at[i,'decision1'] = CHARGE

            if (df.at[i,PRICE] >= ser.quantile(DISCHARGING_PERCENTILE)) & (df.at[i,PRICE] > thelist.mean()):
                df.at[i,'decision1'] = DISCHARGE
            
    return df

In [11]:
def improve(df):
    """ This function tries to modify the decision by increasing the discharging frequency when it's idle """
    
    df['decision2'] = df['decision1']
    
    for i in range(1, len(df)-1):
        
        # if the current decision is idle
        if df.at[i, 'decision1'] == 0:
            
            prev = df.at[i-1, 'decision1']
            next_ = df.at[i+1, 'decision1']
            
            # find the next non-idle decision
            j = i+1
            while next_ == 0:
                j += 1
                next_ = df.at[j, 'decision1']
                
            # if the previous decision and the next non-idle decision is the same
            if next_ == prev:
                
                prices = list(df.loc[i:j-1, PRICE])
                mean_price = sum(prices)/len(prices)
                next_price = df.at[j, PRICE]
                
                # Charge at negative prices
                if (mean_price <= 0) & (mean_price < next_price):
                    df.at[i:j-1, 'decision2'] = CHARGE
                    
                else:
                    # Using the revenue formula, the minimum requirement to earn a revenue
                    # by discharging at i and charging at j is MULTIPLE_OF
                    if mean_price > (MULTIPLE_OF*next_price):
                        df.loc[i:j-1, 'decision2'] = DISCHARGE
                
    
    return df

In [12]:
def get_opencap(i, df):
    """Get the opening battery capacity for every 30-minutes interval 
    Input:
        i : Current row in df
        df : DataFrame with 'opening' and 'closing' column
    Return 
        opening_cap : Opening battery capacity. Integer dtype
    
    """

    if i != 1: #Not the first row
        df.at[i,"opening"] = df.at[i-1,"closing"]
    opening_cap = math.ceil(df.at[i, "opening"])
    
    return opening_cap
    
def get_dispatch(rawPower):
    """ Get the power dispatched for every 30-minutes interval 
    Input:
        rawPower : Current rawPower. Integer Datatype
    Return:
        dispatch : Raw_power dispatched from the market. Integer dtype
    """
    if rawPower < 0:
        eff = 1
    else:
        eff = DISCHARGE_EFF / 100
            
    dispatch = round(rawPower / 2 * eff, 0)
    return dispatch

def get_closecap(opening_cap, dispatch):
    """Get the battery closing capacity for every 30-minutes interval
    Input:
        opening_cap : Opening battery capacity. Integer dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        closecap : Closing battery capacity. Integer dtype
    """
    if dispatch < 0:
        eff = CHARGE_EFF / 100
    else:
        eff = 100 / DISCHARGE_EFF

    closecap = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))
    return closecap
    
def get_revenue(price, dispatch):
    """ Get the Revenue for every 30-minutes interval
    Input:
        price : Market spot price for electricity. Float dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        revenue : current revenue. Integer dtype
    """
    if dispatch < 0:
        factor = 1/MLF
    else:
        factor = MLF 
    revenue = round(price * dispatch * factor)
    return revenue

def run_algo3(i, df, opening_cap):
    """ Get the Raw Power for every 30-minutes interval
    Input:
        i : Current row in df
        df : DataFrame with 'charge_forecaset' and 'discharge_forecast' column
    Return:
        opening_cap : Opening battery capacity. Integer dtype
    """
    raw_power = 0
    if (df.at[i,'decision2'] == CHARGE):
        raw_power = -min(POWER, (CAPACITY-opening_cap)/EFFICIENCY*2)     # CHARGE
        
    elif (df.at[i,'decision2'] == DISCHARGE):
        raw_power = min(POWER, opening_cap/EFFICIENCY*2)      # DISCHARGE
            
    return raw_power

def calculate(df):
    """ Calculate the Battery Opening, Closing Capacity, Raw Power, Market dispatch 
        and Revenue for the entire df.
    Input:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    Return :
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    """
    # Go through each 30-minute interval of df
    for i in list(df.index):
        # get current Spot Price
        price = df.at[i, PRICE]
        
        # update opening capacity
        opening_cap = get_opencap(i, df)

        # find raw_power
        rawPower = run_algo3(i, df, opening_cap)
        df.at[i,'raw_power'] = rawPower

        # find market_dispatch 
        dispatch = get_dispatch(rawPower)
        df.at[i,"dispatch"] = dispatch

        # find closing_capacity   
        df.at[i,"closing"] = get_closecap(opening_cap, dispatch)

        #find revenue        
        df.at[i,"revenue"] = get_revenue(price, dispatch)
    
    return df

In [13]:
def show_result(df):
    """ Print the revenue related information computed from df.
    Input: 
        df : DataFrame with 'revenue' column.
    """
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    
    return None

def run_all(ori_df):
    """ Run the entire data pipeline including initialisng (Data Processing), 
    finding the optimal charging and discarging period (Data Modelling), 
    calculating the revenue based on the optimal period mentioned above (Model Testing and Evaluation).
    
    Input:
        ori_df : DataFrame which contains spot_price for every 30-minute interval. 
    Return:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column.
    """
    # Start time
    start = time.time()
    # Initialise df
    df2 = create_df(ori_df)
    
    # Find Optimal Charging and Discharging period
    df3 = algorithm3(df2)
    
    df3 = improve(df3)
    
    # Calculate the revenue
    df = calculate(df3)
    # Show the revenue       
    show_result(df)
    # End Time      
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [42]:
# Improved
# $93683.48/day
MULTIPLE_OF = 1.58
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.32
DISCHARGING_PERCENTILE = 0.74
test_run = run_all(vic_data)

Total revenue in the dataset: 123849558
Total days in the dataset: 1322.0
Revenue per day: 93683.4780635401
Time Complexity for running the entire Algorithm 3: 29.58071780204773s


In [50]:
i = [0.33]
j = [0.73]
# 94202.74/day -> 0.33, 0.73
charge = np.arange(0.25, 0.36, 0.01)
discharge = np.arange(0.65, 0.78, 0.01)
revenue=[]
highest=0
charge_percentile=0
discharge_percentile=0
for i in charge:
    for q in discharge:
        LOOKAHEAD = 10
        CHARGING_PERCENTILE = i
        DISCHARGING_PERCENTILE = q
        vic_price = run_all(vic_data)
        revenue.append(vic_price["revenue"].sum())
        if(vic_price["revenue"].sum()>highest): 
            highest=vic_price["revenue"].sum()
            charge_percentile=i
            discharge_percentile=q

Total revenue in the dataset: 122355541
Total days in the dataset: 1322.0
Revenue per day: 92553.35930408472
Time Complexity for running the entire Algorithm 3: 28.746270418167114s
Total revenue in the dataset: 122708447
Total days in the dataset: 1322.0
Revenue per day: 92820.30786686839
Time Complexity for running the entire Algorithm 3: 28.900820016860962s
Total revenue in the dataset: 122866423
Total days in the dataset: 1322.0
Revenue per day: 92939.80559757943
Time Complexity for running the entire Algorithm 3: 30.58078098297119s
Total revenue in the dataset: 123032854
Total days in the dataset: 1322.0
Revenue per day: 93065.69894099848
Time Complexity for running the entire Algorithm 3: 30.966729164123535s
Total revenue in the dataset: 122841100
Total days in the dataset: 1322.0
Revenue per day: 92920.65052950075
Time Complexity for running the entire Algorithm 3: 30.872364044189453s
Total revenue in the dataset: 122889976
Total days in the dataset: 1322.0
Revenue per day: 92957

Total revenue in the dataset: 124113795
Total days in the dataset: 1322.0
Revenue per day: 93883.35476550681
Time Complexity for running the entire Algorithm 3: 30.73576259613037s
Total revenue in the dataset: 124048400
Total days in the dataset: 1322.0
Revenue per day: 93833.88804841149
Time Complexity for running the entire Algorithm 3: 29.991007328033447s
Total revenue in the dataset: 123383947
Total days in the dataset: 1322.0
Revenue per day: 93331.276096823
Time Complexity for running the entire Algorithm 3: 29.18342685699463s
Total revenue in the dataset: 123297107
Total days in the dataset: 1322.0
Revenue per day: 93265.58774583964
Time Complexity for running the entire Algorithm 3: 29.119567394256592s
Total revenue in the dataset: 122871576
Total days in the dataset: 1322.0
Revenue per day: 92943.7034795764
Time Complexity for running the entire Algorithm 3: 29.279998064041138s
Total revenue in the dataset: 121512622
Total days in the dataset: 1322.0
Revenue per day: 91915.750

Total revenue in the dataset: 124017791
Total days in the dataset: 1322.0
Revenue per day: 93810.73449319214
Time Complexity for running the entire Algorithm 3: 29.522753715515137s
Total revenue in the dataset: 124139962
Total days in the dataset: 1322.0
Revenue per day: 93903.1482602118
Time Complexity for running the entire Algorithm 3: 30.044286966323853s
Total revenue in the dataset: 124287255
Total days in the dataset: 1322.0
Revenue per day: 94014.56505295007
Time Complexity for running the entire Algorithm 3: 29.9562566280365s
Total revenue in the dataset: 124067276
Total days in the dataset: 1322.0
Revenue per day: 93848.16641452345
Time Complexity for running the entire Algorithm 3: 29.061196088790894s
Total revenue in the dataset: 124097567
Total days in the dataset: 1322.0
Revenue per day: 93871.07942511346
Time Complexity for running the entire Algorithm 3: 29.014655590057373s
Total revenue in the dataset: 124700339
Total days in the dataset: 1322.0
Revenue per day: 94327.0

Total revenue in the dataset: 124375804
Total days in the dataset: 1322.0
Revenue per day: 94081.54614220877
Time Complexity for running the entire Algorithm 3: 29.342502117156982s
Total revenue in the dataset: 123691788
Total days in the dataset: 1322.0
Revenue per day: 93564.13615733737
Time Complexity for running the entire Algorithm 3: 30.6185040473938s
Total revenue in the dataset: 123592529
Total days in the dataset: 1322.0
Revenue per day: 93489.05370650529
Time Complexity for running the entire Algorithm 3: 31.285233974456787s
Total revenue in the dataset: 123173274
Total days in the dataset: 1322.0
Revenue per day: 93171.91679273828
Time Complexity for running the entire Algorithm 3: 31.13757014274597s
Total revenue in the dataset: 121766355
Total days in the dataset: 1322.0
Revenue per day: 92107.68154311649
Time Complexity for running the entire Algorithm 3: 31.214491367340088s


In [43]:
test_run.iloc[1296:1344]

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,decision1,decision2
1297,2018-01-28 00:30:00,79.86,0,0,0,0,0,1,1
1298,2018-01-28 01:00:00,70.01,0,0,0,0,0,1,1
1299,2018-01-28 01:30:00,64.20,0,0,0,0,0,1,1
1300,2018-01-28 02:00:00,70.39,0,0,0,0,0,1,1
1301,2018-01-28 02:30:00,61.22,0,0,0,0,0,1,1
1302,2018-01-28 03:00:00,59.97,0,0,0,0,0,0,0
1303,2018-01-28 03:30:00,59.72,0,0,0,0,0,0,0
1304,2018-01-28 04:00:00,59.90,0,0,0,0,0,0,0
1305,2018-01-28 04:30:00,61.63,0,0,0,0,0,0,0
1306,2018-01-28 05:00:00,57.10,0,0,0,0,0,0,0


In [155]:
test_run.iloc[816:864]

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,decision1,decision2
817,2018-01-18 00:30:00,66.33,0,0,0,0,0,1,1
818,2018-01-18 01:00:00,64.18,0,0,0,0,0,0,0
819,2018-01-18 01:30:00,63.07,0,0,0,0,0,0,0
820,2018-01-18 02:00:00,67.56,0,0,0,0,0,1,1
821,2018-01-18 02:30:00,65.95,0,0,0,0,0,1,1
822,2018-01-18 03:00:00,63.38,0,0,0,0,0,0,0
823,2018-01-18 03:30:00,62.60,0,0,0,0,0,0,0
824,2018-01-18 04:00:00,61.87,0,0,0,0,0,0,0
825,2018-01-18 04:30:00,59.69,-300,-150,-9034,0,135,-1,-1
826,2018-01-18 05:00:00,66.01,0,0,0,135,135,0,0
